In [1]:
from nltk.tokenize import sent_tokenize

from lib.util.graph_data_loader import GraphDataLoader
from lib.model.graph_classifer import GraphClassifier, GraphLSTMClassifier, GraphTransClassifier
from lib.config.config_loader import ConfigLoader

import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.loader import DataLoader

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tqdm import tqdm

config = ConfigLoader().load_config()
tqdm.pandas()

In [2]:
input_dim = config['models']['input_dim']
hidden_dim_1 = config['models']['hidden_dim_1']
hidden_dim_2 = config['models']['hidden_dim_2']
hidden_dim_3 = config['models']['hidden_dim_3']

In [3]:
df = pd.read_csv('./data/processed/reports_labeled.csv')

df.reset_index(drop=True, inplace=True)
df['sentences'] = df['mda'].progress_apply(lambda x: sent_tokenize(x))

train_df = df[df['year'] <= 2019].copy().reset_index(drop=True)
test_df = df[df['year'] > 2019].copy().reset_index(drop=True)

100%|██████████| 13017/13017 [00:36<00:00, 352.69it/s]


In [4]:
print(f"""
## Length of dataset:
Length of training set: {len(train_df)}
Length of test set: {len(test_df)}

## Distribution of label:
Number of merge in training set: {len(train_df[train_df['label'] == 1]['label'])}
Number of not-merge in training set: {len(train_df[train_df['label'] == 0]['label'])}

Number of merge in test set: {len(test_df[test_df['label'] == 1]['label'])}
Number of not-merge in test set: {len(test_df[test_df['label'] == 0]['label'])}
""")


## Length of dataset:
Length of training set: 11019
Length of test set: 1998

## Distribution of label:
Number of merge in training set: 1874
Number of not-merge in training set: 9145

Number of merge in test set: 302
Number of not-merge in test set: 1696



In [5]:
train_corpus = [sentence for sentences in train_df['sentences'] for sentence in sentences]

vectorizer = TfidfVectorizer(max_features=input_dim, stop_words='english')
vectorizer.fit(train_corpus)

def get_tfidf_embeddings(sentence_list):
    if not type(sentence_list) == list:
        sentence_list = [sentence_list]
    embeddings = vectorizer.transform(sentence_list)
    return embeddings

print("Train Sentence: ")
train_df['tfidf_sentence'] = train_df['sentences'].progress_apply(get_tfidf_embeddings)
print("Test Sentence: ")
test_df['tfidf_sentence'] = test_df['sentences'].progress_apply(get_tfidf_embeddings)

Train Sentence: 


100%|██████████| 11019/11019 [00:42<00:00, 259.11it/s]


Test Sentence: 


100%|██████████| 1998/1998 [00:07<00:00, 274.10it/s]


In [6]:
train_dataset = GraphDataLoader(train_df, 5)
test_dataset = GraphDataLoader(test_df, 5)

In [7]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [8]:
def evaluate(y_true, y_pred):
    accuracy = round(accuracy_score(y_true, y_pred), 4)
    precision = round(precision_score(y_true, y_pred, zero_division=0), 4)
    recall = round(recall_score(y_true, y_pred, zero_division=0), 4)
    f1 = round(f1_score(y_true, y_pred, zero_division=0), 4)

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    return accuracy, precision, recall, f1, tp, tn, fp, fn

def train():
    model.train()
    total_loss = 0
    # train_loader üzerinden geçerken progress bar ekleniyor.
    for data, label in tqdm(train_loader, desc="Training", leave=False):
        label = label.squeeze_(1).to(device)
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)  # Modelin çıktısı (logits)
        loss = criterion(out, label)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

# Test fonksiyonu
def test(loader):
    model.eval()
    correct = 0

    y_pred = []
    y_true = []

    for data, label in tqdm(loader, desc="Testing", leave=False):
        label = label.squeeze_(1).to(device)
        data = data.to(device)
        with torch.no_grad():
            out = model(data)
            pred = out.argmax(dim=1)  # En yüksek logit değerine sahip sınıf

            y_true.extend(label.cpu().numpy())
            y_pred.extend(pred.cpu().numpy())
    return y_true, y_pred

In [ ]:
result_dict = {
    "Model": [],
    "Accuracy": [],
    "Precision": [],
    "Recall": [],
    "F1": [],
    "TP": [],
    "TN": [],
    "FP": [],
    "FN": []
}

for model_name in ['GraphClassifier', 'GraphLSTMClassifier', 'GraphTransClassifier']:
    print("Training Model: ", model_name)
    # Model, optimizer ve loss function tanımlanıyor
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    if model_name == 'GraphClassifier':
        model = GraphClassifier(input_dim=input_dim, hidden_dim_1=hidden_dim_1, hidden_dim_2=hidden_dim_2, output_dim=2).to(device)
    elif model_name == 'GraphLSTMClassifier':
        model = GraphLSTMClassifier(input_dim=input_dim, hidden_dim_1=hidden_dim_1, hidden_dim_2=hidden_dim_2, lstm_hidden_dim=hidden_dim_3, output_dim=2).to(device)
    elif model_name == 'GraphTransClassifier':
        model = GraphTransClassifier(input_dim=input_dim, hidden_dim_1=hidden_dim_1, hidden_dim_2=hidden_dim_2, nhead=4, num_encoder_layers=2, output_dim=2).to(device)

    optimizer = optim.Adam(model.parameters(), lr=0.0002)
    criterion = nn.CrossEntropyLoss()

     #Eğitim döngüsü
    num_epochs = 10
    for epoch in range(1, num_epochs + 1):
        loss = train()
        y_true_test, y_pred_test  = test(test_loader)
        accuracy, precision, recall, f1, tp, tn, fp, fn = evaluate(y_true_test, y_pred_test)

        if epoch % 3 == 0 or epoch == num_epochs or epoch == 1:
            print(f"Epoch: {epoch:02d}, Loss: {loss:.4f} | Test | Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1: {f1} | TP: {tp}, TN: {tn}, FP: {fp}, FN: {fn}")

    torch.save(model.state_dict(), f'./outputs/{model_name}.pth')
    result_dict['Model'].append(model_name)
    result_dict['Accuracy'].append(accuracy)
    result_dict['Precision'].append(precision)
    result_dict['Recall'].append(recall)
    result_dict['F1'].append(f1)
    result_dict['TP'].append(tp)
    result_dict['TN'].append(tn)
    result_dict['FP'].append(fp)
    result_dict['FN'].append(fn)

result_df = pd.DataFrame(data=result_dict)
result_df.head()

result_df.to_csv('./outputs/MergerGraphNet.csv', index=False)


Training Model:  GraphClassifier


Epoch: 01, Loss: 0.4540 | Test | Accuracy: 0.8493, Precision: 1.0, Recall: 0.0033, F1: 0.0066 | TP: 1, TN: 1696, FP: 0, FN: 301


Training:  64%|██████▍   | 443/689 [05:44<05:24,  1.32s/it]

In [ ]:
# Eğitim döngüsü
num_epochs = 10
for epoch in range(1, num_epochs + 1):
    loss = train()
    y_true_test, y_pred_test  = test(test_loader)
    accuracy, precision, recall, f1, tp, tn, fp, fn = evaluate(y_true_test, y_pred_test)

    # if epoch % 5 == 0 or epoch == num_epochs or epoch == 1:
    print(f"Epoch: {epoch:02d}, Loss: {loss:.4f} | Test | Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1: {f1} | TP: {tp}, TN: {tn}, FP: {fp}, FN: {fn}")

In [ ]:
result_dict = {
    "Accuracy": [accuracy],
    "Precision": [precision],
    "Recall": [recall],
    "F1": [f1],
    "TP": [tp],
    "TN": [tn],
    "FP": [fp],
    "FN": [fn]
}

result_df = pd.DataFrame(data=result_dict)


In [ ]:
result_df

In [ ]:
result_df.to_csv('./outputs/MergerGraphNet.csv', index=False)
torch.save(model.state_dict(), f'./outputs/MergerGraphNet.pth')